In [2]:
# J. Strong 10/25/2019
# This Python script identifies similar "clusters" of PBI reports using the column names in each file

In [3]:
import pandas as pd
import pyodbc
from sklearn.cluster import KMeans

from datetime import datetime

In [4]:
# Import column name data from SQL Server database
sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                            SERVER=sql2377-fm1-in.amr.corp.intel.com,3180;\
                            DATABASE=SCDA;\
                            Trusted_Connection=yes;\
                            integrated security=true')

query= """SELECT DISTINCT ReportServerType, ReportEnvironment as [Server],FilePath,ColumnName FROM [SCDA].[Audit].[PowerBIColumn]
where TableName NOT LIKE '%LocalDateTable%' AND TableName NOT LIKE '%DateTableTemplate%'"""
df = pd.read_sql(query, sql_conn)
sql_conn.close()

In [5]:
# Use fuzzy word match to determine which column names are nearly the same for all column combinations. The fuzzy match
# can be modified to a higher or lower ratio to control the threshold for "similar" words.

from fuzzywuzzy import fuzz
import itertools

listofsimilar = []

A = df['ColumnName'].str.strip().tolist()
# remove duplicates in list
A = list(dict.fromkeys(A))

# get all combinations of column pairs to be used in fuzzy match comparison
cnt = 0
runningCnt = 0
myIterable = itertools.combinations(A, r=2)
totalCnt = sum(1 for _ in myIterable)

# Run fuzzy logic against all combinations of column names
for element in itertools.combinations(A, r=2):
    tempList = []
    #print(element,fuzz.ratio(element[0],element[1]))
    tempList.append(element)
    ratio = fuzz.ratio(element[0],element[1])
    tempList.append(ratio)
    if ratio >= 90:
        listofsimilar.append(tempList)
    cnt+=1
    if (cnt == 1000000):
        runningCnt += cnt
        print (str(runningCnt) + '/' + str(totalCnt) + ' ' + str(runningCnt/totalCnt))
        cnt = 0

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


1000000/40865320 0.024470626927673635
2000000/40865320 0.04894125385534727
3000000/40865320 0.07341188078302091
4000000/40865320 0.09788250771069454


KeyboardInterrupt: 

In [6]:
similarColumnsdf = pd.DataFrame(listofsimilar, columns =['Columns','Ratio'])
similarColumnsdf = similarColumnsdf.Columns.apply(pd.Series) \
    .merge(similarColumnsdf, left_index = True, right_index = True) \
    .drop(['Columns'], axis = 1)
similarColumnsdf.columns = ['Column1', 'Column2','Ratio']
similarColumnsdf
#similarColumnsdf.to_csv(r'C:\Users\jrstrong\Desktop\temp\SimilarColumns.csv', index=False)
listofsimilar = list(zip(similarColumnsdf.Column1.tolist(),similarColumnsdf.Column2.tolist()))


# Merge all common column names into single list
# https://stackoverflow.com/questions/4842613/merge-lists-that-share-common-elements

LL = set(itertools.chain.from_iterable(listofsimilar)) 
# LL is {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'k', 'o', 'p'}

for each in LL:
  components = [x for x in listofsimilar if each in x]
  for i in components:
    listofsimilar.remove(i)
  listofsimilar += [list(set(itertools.chain.from_iterable(components)))]

In [7]:
# Join list of similar columns to original dataframe

def addColumnKey(a):
    for lst in listofsimilar:
        for word in lst:
            if word == a.strip():
                return sorted(lst)

#%timeit df['ColumnKey'] = df.ColumnName.apply(addColumnKey)
df['ColumnKey_tmp'] = df.ColumnName.apply(addColumnKey)

def updateColumnKey (a,b):
    if b is None:
        return a.strip()
    else:
        return ','.join(b)
        
df['ColumnKey'] = df.apply(lambda x: updateColumnKey(x['ColumnName'], x['ColumnKey_tmp']), axis=1)
df = df.drop("ColumnKey_tmp", axis=1)

In [8]:
#update categoryical variables to "dummy" variables
cat_vars=['ColumnKey']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    df=df.join(cat_list)

#remove columns from dataframe that have been converted to dummies
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
df = df[to_keep]

In [9]:
g = df.groupby(['ReportServerType','Server','FilePath'])
dfGrouped = g.sum()

clmns = list(dfGrouped)

In [10]:
#Cluster the data
k = int(round(len(dfGrouped)*0.75)) # Use 75% of total PBI files as optimal k
kmeans = KMeans(n_clusters=k, random_state=0).fit(dfGrouped)
labels = kmeans.labels_

#Glue back to original data
dfGrouped['clusters'] = labels

#Add the column into our list
clmns.extend(['clusters'])

# drop indexes so dataframe will insert properly into SQL Server table
dfGrouped = dfGrouped[['clusters']].reset_index()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (278) found smaller than n_clusters (288). Possibly due to duplicate points in X.
  return_n_iter=True)


In [11]:
# Load cluster data into SQL Server table

LoadDT = datetime.today().strftime('%Y-%m-%d')

sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                            SERVER=sql2377-fm1-in.amr.corp.intel.com,3180;\
                            DATABASE=SCDA;\
                            Trusted_Connection=yes;\
                            integrated security=true')

cursor = sql_conn.cursor()
cursor.execute('TRUNCATE TABLE [SCDA].[Audit].[ColumnClusters]')

for index,row in dfGrouped.iterrows():
    cursor.execute('INSERT INTO [SCDA].[Audit].[ColumnClusters]([ReportServerType],[Server],[FilePath],[clusters]) VALUES (?,?,?,?)', row[0],row[1],row[2],row[3])
    sql_conn.commit()
    
sql = "UPDATE [SCDA].[Audit].[ColumnClusters] SET [LastUpdate] = ?"
params = (LoadDT)

cursor.execute(sql,params)
sql_conn.commit()
sql_conn.close()